# B01. Matchups
- Create matchup files with batter and pitcher stats for both the home and away teams
- Files are unimputed

In [1]:
%run "U1. Imports.ipynb"
%run "U2. Utilities.ipynb"
%run "U3. Classes.ipynb"

baseball_path = r'C:\Users\james\Documents\MLB\Database'

db_path = r'C:\Users\james\Documents\MLB\Database\MLBDB.db'
engine = create_engine(f'sqlite:///{db_path}')

In [2]:
%run "A02. MLB API.ipynb"
%run "A03. Steamer.ipynb"

### Models

##### Inputs

In [3]:
# Batters
with open(os.path.join(model_path, "batter_stats_scaler_20231027.pkl"), "rb") as file:
    batter_stats_scaler = pickle.load(file)
# Pitchers
with open(os.path.join(model_path, "pitcher_stats_scaler_20231027.pkl"), "rb") as file:
    pitcher_stats_scaler = pickle.load(file)

##### FanGraphs

In [4]:
# Batters
with open(os.path.join(model_path, "batter_stats_fg_scaler_20231027.pkl"), "rb") as file:
    batter_stats_fg_scaler = pickle.load(file)
# Pitchers
with open(os.path.join(model_path, "pitcher_stats_fg_scaler_20231027.pkl"), "rb") as file:
    pitcher_stats_fg_scaler = pickle.load(file)

### Clean Datasets

In [5]:
# Clean draftable players 
def clean_draftables(draftGroupId):
    # Read in draftables
    sql_query = f'''
    SELECT *
    FROM "Draftables {draftGroupId}"
    '''

    draftables = pd.read_sql_query(sql_query, con=engine)
    
    # Replace @ sign for easier splitting
    draftables['Game Info2'] = draftables['Game Info'].str.replace("@", " ")
    # Extract game info
    draftables[['Away', 'Home', 'date_slash', 'time', 'zone']] = draftables['Game Info2'].str.split(' ', 4, expand=True)
    
    # Convert the "time" column to datetime
    draftables['datetime'] = pd.to_datetime(draftables['date_slash'] + ' ' + draftables['time'], format='%m/%d/%Y %I:%M%p')
    # Sort the DataFrame by the "time" column
    draftables = draftables.sort_values(by='datetime')

    # Clean name
    draftables['Name'] = draftables['Name'].apply(lambda x: unidecode.unidecode(x))

    # Extract unique "Game Info" values and store them in a list
    matchups = draftables['Game Info'].unique().tolist()
    
    return draftables, matchups

In [6]:
# Extract game information such as gamePk
def game_info(matchup, games, team_map):
    # Determine home and away team
    parts = matchup.split()
    away, home = parts[0].split('@')
    # Retrieve home team ID
    home_id = team_map.loc[home, 'teamId']
    # Determine date
    date_slash = parts[1]  
    date_dash = date_slash[6:] + "-" + date_slash[:2] + "-" + date_slash[3:5]
    # Determine datetime 
    dt = date_dash + " " + parts[2] + " " + parts[3]
    dt = parser.parse(dt, tzinfos={"ET": "US/Eastern"})
    # Create a new datetime object representing 6:00 PM EST
    est = pytz.timezone("US/Eastern")
    six_pm_est = est.localize(datetime.datetime(dt.year, dt.month, dt.day, 18, 0))
    # Check if the parsed datetime is later than 6:00 PM EST
    # If it is, and it's a doubleheader, it's the late game. Else, it's the early game.
    if dt > six_pm_est:
        late = True
    else:
        late = False
    
    gamePk = None
    # Loop over all games
    for game in games:   
        # If date and home team match
        if game['game_date'] == date_dash and game['home_id'] == home_id:
            # And it's a doubleheader
            if game['doubleheader'] == "Y":
                # If it's late in the day and it's a double header, it's game two
                if late and game['game_num'] == 2:
                    gamePk = game['game_id']
                # Else, it's game 1
                elif not late and game['game_num'] == 1:
                    gamePk = game['game_id']
            # If it's not a double header, there will only be one.
            elif game['doubleheader'] == "N":
                gamePk = game['game_id']
            # Identify venue id
            venue_id = game['venue_id']     
            # Away starter
            away_starter = game['away_probable_pitcher']
            # Home starter
            home_starter = game['home_probable_pitcher']
                
                
    return date_slash, gamePk, venue_id, away, home, away_starter, home_starter

In [7]:
# Create player dataframes
def create_matchup_file(dkteam, date_slash, gamePk, venue_id, away_starter, home_starter, draftables, team_map, complete_dataset, steamer_hitters, steamer_pitchers):
    # Retrieve Baseball Reference team abbreviation
    team = team_map.loc[dkteam, 'BBREFTEAM']
        
    # Convert date to compatible format
    date = date_slash[6:] + date_slash[:2] + date_slash[3:5]
    
    ### Roster
    sql_query = f'''
    SELECT *
    FROM "Roster {team} {date}"
    '''
    roster = pd.read_sql_query(sql_query, con=engine)
    
    
    ### Batting order
    sql_query = f'''
    SELECT *
    FROM "Batting Order {team} {gamePk}"
    '''
    order = pd.read_sql_query(sql_query, con=engine)
    # Would want to fix order here for upcoming games without order variable
    
    
    ### Bullpen
    sql_query = f'''
    SELECT *
    FROM "Bullpen {team} {date}"
    '''   
    bullpen = pd.read_sql_query(sql_query, con=engine)
    bullpen.rename(columns={'Name': 'fullName'}, inplace=True)

        
    # Merge batting order onto roster
    team_df = pd.merge(roster, order[['id', 'status', 'order']], on='id', how='left')
    # Merge pitcher leverage onto roster
    team_df = pd.merge(team_df, bullpen[['fullName', 'Leverage']], on='fullName', how='left')
    # Merge draftables
    team_df = pd.merge(team_df, draftables[['Name + ID', 'Name', 'ID', 'playerId', 'Position', 'Roster Position', 'Salary', 'AvgPointsPerGame']], left_on='fullName', right_on='Name', how='left')
    
    # Add weather
    box = create_box(gamePk)
    team_df['weather'] = box[0]
    team_df['wind'] = box[1]
    team_df['park'] = box[2]
    team_df = clean_weather(team_df)
    
    # Add venue
    team_df['venue_id'] = venue_id
    
    # Add starters
    team_df['away_starter'] = away_starter
    team_df['home_starter'] = home_starter
    
    # Assign Leverage of 1 to starting pitcher
    team_df['Leverage'] = np.where((team_df['fullName'] == team_df['away_starter']) | (team_df['fullName'] == team_df['home_starter']), 1, team_df['Leverage'])
    
    # Determine batting order
    team_df['batting_order'] = np.nan
    for i in range(9):
        team_df['batting_order'] = np.where(team_df['order'] == (i+1)*100, i+1, team_df['batting_order'])
    
    ### Batters
    batter_df = team_df[team_df['position'] != "Pitcher"]
    
    ## Dataset
    # Vs. LHP
    vs_l = complete_dataset[complete_dataset['date'] < int(date)]
    vs_l = vs_l[vs_l['pitchHand'] == "L"]
    vs_l.drop_duplicates(subset='batter', keep='last', inplace=True)
    
    # Merge in stats
    batter_df = pd.merge(batter_df, vs_l[['batter'] + batter_stats + ['imp_b']], left_on='id', right_on='batter', how='left')
    
    # Vs. RHP
    vs_r = complete_dataset[complete_dataset['date'] < int(date)]
    vs_r = vs_r[vs_r['pitchHand'] == "R"]
    vs_r.drop_duplicates(subset='batter', keep='last', inplace=True)
    
    # Merge in stats
    batter_df = pd.merge(batter_df, vs_r[['batter'] + batter_stats + ['imp_b']], left_on='id', right_on='batter', how='left', suffixes=("_l", "_r"))
    
    ## Steamer 
    # Keep last observation before date (may switch to <= if I find projections are up early)
    steamer_hitters_last_df = steamer_hitters[steamer_hitters['date'] < int(date)]
    steamer_hitters_last_df.drop_duplicates(subset='mlbamid', keep='last', inplace=True)

    # Merge
    batter_df = pd.merge(batter_df, steamer_hitters_last_df, left_on='id', right_on='mlbamid', how='left', suffixes=("", "_fg"))
    
    # Remove redundant variables
    batter_df.drop(columns={'batter_l', 'batter_r', 'firstname', 'lastname', 'mlbamid'}, inplace=True)
    
    # Sort
    batter_df.sort_values('batting_order', inplace=True)
    
    
    ### Pitchers
    pitcher_df = team_df[(team_df['position'] == "Pitcher") | (team_df['position'] == "Two-Way Player")]
    
    ## Dataset
    # Vs. LHB
    vs_l = complete_dataset[complete_dataset['date'] < int(date)]
    vs_l = vs_l[vs_l['batSide'] == "L"]
    vs_l.drop_duplicates(subset='pitcher', keep='last', inplace=True)
    
    # Merge in stats
    pitcher_df = pd.merge(pitcher_df, vs_l[['pitcher'] + pitcher_stats + ['imp_p']], left_on='id', right_on='pitcher', how='left')
    
    # Vs. RHB
    vs_r = complete_dataset[complete_dataset['date'] < int(date)]
    vs_r = vs_r[vs_r['batSide'] == "R"]
    vs_r.drop_duplicates(subset='pitcher', keep='last', inplace=True)
    
    # Merge in stats
    pitcher_df = pd.merge(pitcher_df, vs_r[['pitcher'] + pitcher_stats + ['imp_p']], left_on='id', right_on='pitcher', how='left', suffixes=("_l", "_r"))
    
    ## Steamer 
    # Keep last observation before date (may switch to <= if I find projections are up early)
    steamer_pitchers_last_df = steamer_pitchers[steamer_pitchers['date'] < int(date)]
    steamer_pitchers_last_df.drop_duplicates(subset='mlbamid', keep='last', inplace=True)

    # Merge
    pitcher_df = pd.merge(pitcher_df, steamer_pitchers_last_df, left_on='id', right_on='mlbamid', how='left', suffixes=("", "_fg"))
    
    # Remove redundant variables
    pitcher_df.drop(columns={'pitcher_l', 'pitcher_r', 'firstname', 'lastname', 'mlbamid'}, inplace=True)
    
    # Sort
    pitcher_df.sort_values('Leverage', inplace=True)
    
    return batter_df, pitcher_df

### Read Datasets

In [8]:
# Read in API/Statcast data
def read_dataset():
    # Choose the last instance of each player in each game, assuming they have enough PAs
    sql_query = f'''
        SELECT *
        FROM "Dataset"
        WHERE date >= 20220301
        '''

    df = pd.read_sql_query(sql_query, con=engine)
    
    return df

In [9]:
# Read in Steamer hitter data
def read_steamer_hitters():
    # Choose the last instance of each player in each game, assuming they have enough PAs
    sql_query = f'''
        SELECT *
        FROM "Steamer Hitters"
        WHERE proj_year >= 2022
        '''

    df = pd.read_sql_query(sql_query, con=engine)
    df2 = clean_steamer_hitters(df)
    
    return df2

In [10]:
# Read in steamer pitcher data
def read_steamer_pitchers():
    # Choose the last instance of each player in each game, assuming they have enough PAs
    sql_query = f'''
        SELECT *
        FROM "Steamer Pitchers"
        WHERE proj_year >= 2022
        '''

    df = pd.read_sql_query(sql_query, con=engine)
    df2 = clean_steamer_pitchers(df)
    
    return df2

In [11]:
# Read in team_map
def read_team_map():
    # Choose the last instance of each player in each game, assuming they have enough PAs
    sql_query = f'''
        SELECT DKTEAM, BBREFTEAM, teamId
        FROM "Team Map"
        '''

    team_map = pd.read_sql_query(sql_query, con=engine)
    team_map.set_index('DKTEAM', inplace=True)
    
    return team_map

In [12]:
# Read in schedule to extract gamePKs based on teams and dates
def read_schedule():
    # 2022 games
    games2022 = statsapi.schedule(start_date="03/04/2022", end_date="11/15/2022")
    # 2023 games
    games2023 = statsapi.schedule(start_date="03/04/2023", end_date=todaysdate_slash)
    # Add 'em together
    games = games2022 + games2023
    
    return games

### Read in Datasets
This should likely go in a dashboard

In [13]:
# Read in API/Statcast data
complete_dataset = read_dataset()
# Read in Steamer hitters
steamer_hitters_df = read_steamer_hitters()
# Read in Steamer pitchers
steamer_pitchers_df = read_steamer_pitchers()
# Read in team map
team_map = read_team_map()
# Read in all games since 2022
games = read_schedule()

In [14]:
draftGroupId = 85143

In [15]:
# Clean draftables file and extract matchups
draftables, matchups = clean_draftables(draftGroupId)

In [16]:
for matchup in matchups:
    print(matchup)
    # Extract info from matchup
    date_slash, gamePk, venue_id, away, home, away_starter, home_starter = game_info(matchup, games, team_map)
    # Away team
    away_batter_df, away_pitcher_df = create_matchup_file(away, date_slash, gamePk, venue_id, away_starter, home_starter, draftables, team_map, complete_dataset, steamer_hitters_df, steamer_pitchers_df)
    # Home team
    home_batter_df, home_pitcher_df = create_matchup_file(home, date_slash, gamePk, venue_id, away_starter, home_starter, draftables, team_map, complete_dataset, steamer_hitters_df, steamer_pitchers_df)
    
    # Create folder, if it doesn't exist
    try:
        os.mkdir(os.path.join(baseball_path, "B01. Matchups", f'Matchups {draftGroupId}'))
    except:
        pass

    # File name
    matchup_file = matchup.replace('/', '').replace(':', '')
    
    # Write to Excel
    away_batter_df.to_excel(os.path.join(baseball_path, "B01. Matchups", f'Matchups {draftGroupId}', f'{matchup_file}.xlsx'), sheet_name="AwayBatters", engine='openpyxl', index=False)

    with pd.ExcelWriter(os.path.join(baseball_path, "B01. Matchups", f'Matchups {draftGroupId}', f'{matchup_file}.xlsx'), mode='a', engine='openpyxl') as writer:  
        home_batter_df.to_excel(writer, sheet_name='HomeBatters', index=False)
        
    with pd.ExcelWriter(os.path.join(baseball_path, "B01. Matchups", f'Matchups {draftGroupId}', f'{matchup_file}.xlsx'), mode='a', engine='openpyxl') as writer:  
        away_pitcher_df.to_excel(writer, sheet_name='AwayPitchers', index=False)

    with pd.ExcelWriter(os.path.join(baseball_path, "B01. Matchups", f'Matchups {draftGroupId}', f'{matchup_file}.xlsx'), mode='a', engine='openpyxl') as writer:  
        home_pitcher_df.to_excel(writer, sheet_name='HomePitchers', index=False)

DET@TOR 04/11/2023 07:07PM ET
SD@NYM 04/11/2023 07:10PM ET
CIN@ATL 04/11/2023 07:20PM ET
CWS@MIN 04/11/2023 07:40PM ET
SEA@CHC 04/11/2023 07:40PM ET
KC@TEX 04/11/2023 08:05PM ET
STL@COL 04/11/2023 08:40PM ET
WAS@LAA 04/11/2023 09:38PM ET
MIL@ARI 04/11/2023 09:40PM ET
LAD@SF 04/11/2023 09:45PM ET


In [17]:
# Extract info from matchup
date_slash, gamePk, venue_id, away, home, away_starter, home_starter = game_info(matchups[2], games, team_map)
date_slash, gamePk, venue_id, away, home, away_starter, home_starter

('04/11/2023', 718620, 4705, 'CIN', 'ATL', 'Luis Cessa', 'Kyle Wright')

In [18]:
# Away team
away_batter_df, away_pitcher_df = create_matchup_file(away, date_slash, gamePk, venue_id, away_starter, home_starter, draftables, team_map, complete_dataset, steamer_hitters_df, steamer_pitchers_df)
# Home team1
home_batter_df, home_pitcher_df = create_matchup_file(home, date_slash, gamePk, venue_id, away_starter, home_starter, draftables, team_map, complete_dataset, steamer_hitters_df, steamer_pitchers_df)

In [19]:
home_batter_df['imp_b']

KeyError: 'imp_b'

In [ ]:
home_batter_df[['Name', 'b1_b_r', 'b1_b_long_r']]